In [ ]:
import os
import mat73
import numpy as np
import pandas as pd
from scipy.signal import welch

def get_avg_psds(freqs, psds):
    FREQ_BANDS = {"delta": [1, 5.0],
                  "theta": [5.0, 8.0],
                  "alpha": [8.0, 13.0],
                  "beta": [13.0, 30.0],
                  "gamma": [30.0, 40.0]}

    psds_bands = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_slice = psds[(freqs >= fmin) & (freqs < fmax)]
        if len(psds_slice) > 0:
            psds_band = psds_slice.mean()
        else:
            psds_band = 1  # or any other value you prefer for empty slices
        psds_bands.append(psds_band)

    abs_psds = np.array(psds_bands)  # Absolute band values
    return abs_psds

def save_to_csv(subject_dir, filename, df=None):
    data_dict = mat73.loadmat(os.path.join(subject_dir, 'erp_v1.mat'))
    log_data = data_dict["log"]
    channels_of_interest = [37, 30, 47, 12, 49 , 14 ,51]
    channel_names = {'37': 'Fz', '30': 'Pz', '47': 'Cz', '12': 'C3', '49': 'C4', '14' : 'T7' , '51' : 'T8'}
    column_names = ['Subject', 'Channel', 'Trial', 'Smell_ID', 'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
    df_list = []
    subject_id = os.path.basename(subject_dir)
    for channel_num in channels_of_interest:
        channel_name = channel_names[str(channel_num)]
        for trial in range(data_dict["erp"].shape[2]):
            freqs, psds = welch(data_dict["erp"][channel_num, :, trial], fs=500)
            abs_psds = get_avg_psds(freqs, psds)
            smell_id = log_data[trial, 1]  # Get the smell ID for the current trial
            df_list.append(pd.DataFrame([[subject_id, channel_name, trial + 1, smell_id] + list(abs_psds)], columns=column_names))
    if df is None:
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.concat([df] + df_list, ignore_index=True)
    df.to_csv(filename, index=False)

# Path to the directory containing subject data
subject_data_dir = "C:\Users\Aryan\Documents\MATLAB\Kato_et_al - Copy\Kato_et_al\ProcData\EEG"

# Initialize an empty DataFrame to hold all data
all_data_df = None

# Iterate over subject directories
for subject_dir in os.listdir(subject_data_dir):
    subject_dir_path = os.path.join(subject_data_dir, subject_dir)
    if os.path.isdir(subject_dir_path):
        csv_filename = f"psd_results_all_subjects_1_sec.csv"
        save_to_csv(subject_dir_path, csv_filename, all_data_df)

print("Data saved to psd_results_all_subjects_1_sec.csv")

In [6]:
import os
import mat73
import numpy as np
import pandas as pd
from scipy.signal import welch

def get_avg_psds(freqs, psds):
    FREQ_BANDS = {"delta": [1, 5.0],
                  "theta": [5.0, 8.0],
                  "alpha": [8.0, 13.0],
                  "beta": [13.0, 30.0],
                  "gamma": [30.0, 40.0]}

    psds_bands = []
    for fmin, fmax in FREQ_BANDS.values():
        psds_slice = psds[(freqs >= fmin) & (freqs < fmax)]
        if len(psds_slice) > 0:
            psds_band = psds_slice.mean()
        else:
            psds_band = 1  # or any other value you prefer for empty slices
        psds_bands.append(psds_band)

    abs_psds = np.array(psds_bands)  # Absolute band values
    return abs_psds

def save_to_csv(subject_dir, filename, df=None):
    data_dict = mat73.loadmat(os.path.join(subject_dir, 'erp_v1.mat'))
    log_data = data_dict["log"]
    channels_of_interest = [37, 30, 47, 12, 49, 14, 51]
    channel_names = {'37': 'Fz', '30': 'Pz', '47': 'Cz', '12': 'C3', '49': 'C4', '14': 'T7', '51': 'T8'}
    column_names = ['Subject', 'Channel', 'Trial', 'Smell_ID', 'Delta', 'Theta', 'Alpha', 'Beta', 'Gamma']
    df_list = []
    subject_id = os.path.basename(subject_dir)
    for channel_num in channels_of_interest:
        channel_name = channel_names[str(channel_num)]
        for trial in range(data_dict["erp"].shape[2]):
            # Adjust the data range to extract samples from 1100 to 1600
            erp_data = data_dict["erp"][channel_num, 1100:1600, trial]
            freqs, psds = welch(erp_data, fs=500)
            abs_psds = get_avg_psds(freqs, psds)
            smell_id = log_data[trial, 1]  # Get the smell ID for the current trial
            df_list.append(pd.DataFrame([[subject_id, channel_name, trial + 1, smell_id] + list(abs_psds)],
                                         columns=column_names))
    if df is None:
        df = pd.concat(df_list, ignore_index=True)
    else:
        df = pd.concat([df] + df_list, ignore_index=True)
    df.to_csv(filename, index=False)


# Path to the directory containing subject data
root_directory = r"C:\Users\Aryan\Documents\MATLAB\Kato_et_al - Copy\Kato_et_al\ProcData\EEG"

# Initialize an empty DataFrame to hold all data
all_data_df = None

# Iterate over subject directories
for subject_dir in os.listdir(root_directory):
    if not subject_dir.startswith('.'):  # Skip directories starting with '.'
        subject_dir_path = os.path.join(root_directory, subject_dir)
        if os.path.isdir(subject_dir_path):
            save_to_csv(subject_dir_path, "psd_results_all_subjects_1_sec.csv", all_data_df)

print("Data saved to psd_results_all_subj")


Data saved to psd_results_all_subj
